# Delta Lake Lab 
## Unit 3: Delta Table Utilities

This lab is powered by Dataproc Serverless Spark.

In the previous units, we covered the below-
1. Create a base delta table off of the parquet base table loan_db.loans_by_state_parquet
2. Take a peek under the hood of the Delta table
3. Review the delta transaction log
4. Look at delta table details
5. Look at delta table history
6. Create a manifest file
7. Review entries in the Hive Metastore (Dataproc Metastore Service)

In this unit, we will -
1. Review Delta table details
2. Review Delta table history
3. Learn how to create a manifest file
4. Review metastore entries
,

### 1. Imports

In [1]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

22/10/22 23:31:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  885979867746


In [ ]:
ACCOUNT_NAME = "akhjain"

In [6]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}-{ACCOUNT_NAME}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"

### 4. Peek under the hood of our Delta Lake table (loan_db.loans_by_state_delta)

In [7]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json


In [8]:
!gcloud storage cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000000.json

{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"56ed94c0-271d-4a3f-b076-4a0aca50c945","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"addr_state\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"count\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1666481414161}}
{"add":{"path":"part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet","partitionValues":{},"size":978,"modificationTime":1666481421062,"dataChange":true,"stats":"{\"numRecords\":51,\"minValues\":{\"addr_state\":\"AK\",\"count\":1},\"maxValues\":{\"addr_state\":\"WY\",\"count\":1},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"commitInfo":{"timestamp":1666481424546,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"1","numOutp

### 5. Table Details
https://docs.delta.io/latest/delta-utility.html#id6

In [9]:
deltaTable = DeltaTable.forPath(spark, DELTA_LAKE_DIR_ROOT)
detailDF = deltaTable.detail()
detailPDF=detailDF.toPandas()
detailPDF

,format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
0,delta,56ed94c0-271d-4a3f-b076-4a0aca50c945,None,None,gs://dll-data-bucket-885979867746/delta-consum...,2022-10-22 23:30:14.161,2022-10-22 23:30:24.912,[],1,978,{},1,2


### 6. Table History

https://docs.delta.io/latest/delta-utility.html#id4

In [10]:
deltaTable = DeltaTable.forPath(spark, DELTA_LAKE_DIR_ROOT)
fullHistoryPDF = deltaTable.history().toPandas()    # get the full history of the table
lastOperationPDF = deltaTable.history(1).toPandas() # get the last operation

#### Last operation

In [11]:
lastOperationPDF

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,0,2022-10-22 23:30:24.912,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '51', 'numOutputBytes': '978...",None,Apache-Spark/3.3.0 Delta-Lake/2.1.0


#### Full History

In [12]:
fullHistoryPDF

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,0,2022-10-22 23:30:24.912,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '51', 'numOutputBytes': '978...",None,Apache-Spark/3.3.0 Delta-Lake/2.1.0


### 7. Table manifest file
https://docs.delta.io/latest/delta-utility.html#id8

You can a generate manifest file for a Delta table that can be used by other processing engines (that is, other than Apache Spark) to read the Delta table. For example, to generate a manifest file that can be used by Presto and Athena to read a Delta table, you run the following:

In [13]:
deltaTable = DeltaTable.forPath(spark, DELTA_LAKE_DIR_ROOT)
deltaTable.generate("symlink_format_manifest")

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used


In [14]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT | grep "_symlink_format_manifest/manifest"

gs://dll-data-bucket-885979867746/delta-consumable/_symlink_format_manifest/manifest


In [15]:
MANIFEST_LIST = !gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT | grep "_symlink_format_manifest/manifest"
MANIFEST_FILE = MANIFEST_LIST[0]
print(MANIFEST_FILE)

gs://dll-data-bucket-885979867746/delta-consumable/_symlink_format_manifest/manifest


In [16]:
!gcloud storage cat $MANIFEST_FILE

gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet


Using this manifest file, you can create an external table in BigQuery on the Delta Table, except it will be point in time to when the manifest was generated.

### 8. Hive Metastore Entry

In [17]:
spark.sql("show tables in loan_db").show(truncate=False)

+---------+----------------------+-----------+
|namespace|tableName             |isTemporary|
+---------+----------------------+-----------+
|loan_db  |loans_by_state_delta  |false      |
|loan_db  |loans_by_state_parquet|false      |
|loan_db  |loans_cleansed_parquet|false      |
+---------+----------------------+-----------+



### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK